In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv

torch.manual_seed(1)

#### Loading Dataset - Preprocessing on the Tweets

In [5]:
def remove_writing_marks(string):
  for char in ['?','!','#','@',',','.','$','%','^','&','*',')','(','/','~','`','&','+','+','_',';',':',"'"]:
    if char == '#':
      replaced = "Hashtag "
    elif  char == '@':
      replaced = "Atsign "
    else:
      replaced = ''
    string = string.replace(char, replaced)
  return string

def change_label(label):
  if '0' in label:
    label = 0
  elif '2' in label:
    label = 1
  elif '4' in label:
    label = 2
  return label

In [7]:
from google.colab import drive
drive.mount('/content/drive')

train_data_dir = r'/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/Datas/train.csv'
test_data_dir = r'/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/Datas/test.csv'
data_train = []
data_test = []
words = []

with open(train_data_dir, 'r') as csvfile:
  csvreader = csv.reader(csvfile)
  try:
    for row in csvreader:
      if 'http' in row[5]:
        continue
      else:
        text = remove_writing_marks(row[5])
        label = change_label(row[0])
        data_train.append([label, text])

        for word in text.split():
          if word in words:
            pass
          else:
            words.append(word)
  except:
      pass

with open(test_data_dir, 'r') as csvfile:
  csvreader = csv.reader(csvfile,)
  try:
    for row in csvreader:
      text = remove_writing_marks(row[5])
      label = change_label(row[0])
      data_test.append([label, text])

      for word in text.split():
          if word in words:
            pass
          else:
            words.append(word)
  except:
    pass


word_to_ix = {word: i for i, word in enumerate(words)}
word_to_ix[' '] = len(words)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Text Embedding

In [22]:
import pickle 
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/words_dict.pkl', 'wb') as f:
    pickle.dump(word_to_ix, f)

with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/data_test.pkl', 'wb') as f:
    pickle.dump(data_test, f)

with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/data_train.pkl', 'wb') as f:
    pickle.dump(data_train, f)

In [41]:
import pickle 
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/words_dict.pkl', 'rb') as f:
    word_to_ix = pickle.load(f)

with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/data_test.pkl', 'rb') as f:
    data_test = pickle.load(f)

with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q01/data_train.pkl', 'rb') as f:
    data_train = pickle.load(f)

In [42]:
embedding_dim = len(words)+1
vocab_size = 38

for data in data_train:
  Size = len(data[1])
  if Size < vocab_size:
    for i in range(vocab_size-Size):
      data[1].append(' ')

for data in data_test:
  data[1] = data[1].split()
  Size = len(data[1])
  if Size < vocab_size:
    for i in range(vocab_size-Size):
      data[1].append(' ')

In [51]:
embedding_dim = len(words)+1

In [68]:
embeddings = nn.Embedding(embedding_dim, vocab_size)

In [76]:
context = data_test[2][1]
context = [word_to_ix[w] for w in context]
context = torch.tensor(context, dtype=torch.long)
embeddings(context)

tensor([[-0.8737,  0.8229, -0.8905,  ..., -0.8210,  0.5535,  0.0424],
        [-0.1408, -1.4420,  0.0322,  ..., -0.2135, -0.5094,  1.7105],
        [ 1.1217, -1.1332,  1.1968,  ..., -0.1908, -0.8057,  0.0472],
        ...,
        [-1.6494, -0.2544, -1.4821,  ..., -0.1532, -0.6081, -1.1262],
        [-1.6494, -0.2544, -1.4821,  ..., -0.1532, -0.6081, -1.1262],
        [-1.6494, -0.2544, -1.4821,  ..., -0.1532, -0.6081, -1.1262]],
       grad_fn=<EmbeddingBackward0>)

In [54]:
embedding_dim

203379